In [ ]:
"""
Colab Notebook Set Up

Use this cell to upload your kaggle.json file as well as the `download_data.sh`
and `preprocess.py` scripts.
"""

from google.colab import files
import os

# Upload files
kaggle = files.upload()
data_script = files.upload()
preprocess_script = files.upload()
config = files.upload()

# Verify uploads
for file in ["kaggle.json", "download_data.sh", "preprocess.py", "oct.yaml"]:
    assert file in os.listdir(), f"Make sure you upload the {file} file"

# Shell commands
!mkdir -p ~/.kaggle/ data/ models/ config/ scripts/
!mv kaggle.json ~/.kaggle/
!mv download_data.sh preprocess.py scripts/
!mv oct.yaml config/
!chmod 600 ~/.kaggle/kaggle.json
!chmod +x scripts/download_data.sh scripts/preprocess.py
!pip install -q kaggle pretrainedmodels rich

!sed -i -e 's/\r$//' scripts/download_data.sh

# Run shell commands
!scripts/download_data.sh
!python scripts/preprocess.py --config config/oct.yaml

In [ ]:
%load_ext tensorboard
%load_ext autoreload
%autoreload 2

%tensorboard --logdir runs

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
import pretrainedmodels as models

from datetime import datetime
from tqdm.auto import tqdm
from rich import print
from glob import glob

import sys; sys.path.append(".")
from net import train

import numpy as np
import torch
import copy
import time
import yaml
import os
import re

config = 'config/oct.yaml'

with open(config, "r") as file:
    config = yaml.safe_load(file)

In [ ]:
class FineTuned(nn.Module):
    def __init__(self, config):
        super(FineTuned, self).__init__()

        self.model = models.__dict__[config["model-name"]](num_classes=1000, pretrained="imagenet")
        for param in self.model.parameters():
            param.requires_grad = False

        self.model.last_linear = nn.Linear(self.model.last_linear.in_features, config["num-classes"])

    def forward(self, x):
        x = self.model(x)
        return x

# Create dataloaders
loaders = train.feature_dataloaders(config)

# Create model and move to GPU
model = FineTuned(config)
model = model.to(train.device())

# Define parameters to optimize
update_params = filter(lambda p: p.requires_grad, model.parameters())

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(update_params, lr=config["lr"])
# optimizer = optim.SGD(update_params, lr=config["lr"], momentum=config["momentum"])

In [ ]:
def train_model(
    model,
    dataloaders,
    criterion,
    optimizer,
    num_epochs=25,
    scheduler=None,
    writer=None,
    verbose=True,
    is_inception=True,
):

    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in tqdm(dataloaders[phase], leave=False):
                inputs = inputs.to(train.device())
                labels = labels.to(train.device())

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    """
                    Inception is a special case during training because it has an auxiliary output.
                    When training, we calculate the loss by summing the final output and the auxiliary output
                    but during testing we only consider the final output.
                    """
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)

                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
            
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history